In [32]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob

In [33]:
img_dir = '/data/handhygiene/images/'
excel_path = os.path.join(os.getcwd(), 'hh_video_data.xlsx')
label_path = os.path.join(os.getcwd(), 'handhygiene_label.csv') 

In [41]:
hh_dict = {'video_id':[], 'date':[], 'img_name':[]}
def create_hh_csv(img_dir):
    for image in tqdm(glob(img_dir+'*.jpg')):
        img_name= image.split('/')[-1].split('.')[0]

        video_id = img_name.split('_')[0]
        date = img_name.split('_')[1]
        frame_num = img_name[-6:]
        
        hh_dict['video_id'].append(video_id)
        hh_dict['date'].append(date)
        hh_dict['img_name'].append(img_name)

In [42]:
create_hh_csv(img_dir)

100%|██████████| 4122/4122 [00:00<00:00, 276644.87it/s]


In [43]:
df = pd.DataFrame(hh_dict)
df.sort_values('img_name', inplace=True)
df.reset_index(inplace=True)
df.drop('index', 1, inplace=True)

In [44]:
df

,date,img_name,video_id
0,20180806,1_20180806_frames000000,1
1,20180806,1_20180806_frames000001,1
2,20180806,1_20180806_frames000002,1
3,20180806,1_20180806_frames000003,1
4,20180806,1_20180806_frames000004,1
5,20180806,1_20180806_frames000005,1
6,20180806,1_20180806_frames000006,1
7,20180806,1_20180806_frames000007,1
8,20180806,1_20180806_frames000008,1
9,20180806,1_20180806_frames000009,1


In [45]:
df_excel = pd.read_excel(excel_path)

In [46]:
hh_dict = {'date':[], 'img_name':[], 'video_id':[], 'target':[] }
temp_array = []
for row in df.values[:]:
    video_id = int(row[2])
    frame_num = row[1].split('_')[-1][-6:]
    for vid, target, length in df_excel[['video_id', 'target_frame', 'frame_length']].values:
        if video_id is not vid:
            continue
        if target is np.nan:
            continue
        
        target_frames = target.strip().split(',')
        target_frames = list(map(lambda x: "%.6d" % int(x) ,target_frames))
    
        if frame_num in target_frames:
            # update target frame list
            temp_array = list(map(lambda n: "%.6d" % (n + int(frame_num)), range(int(length))))
            row = np.append(row, [1])
            continue
        if frame_num in temp_array:
            row = np.append(row, [1])
        else:
            row = np.append(row, [0])
        print(row)   
        hh_dict['date'].append(row[0]) 
        hh_dict['img_name'].append(row[1])
        hh_dict['video_id'].append(row[2])
        hh_dict['target'].append(row[3])

['20180806' '1_20180806_frames000000' '1' 0]
['20180806' '1_20180806_frames000001' '1' 0]
['20180806' '1_20180806_frames000002' '1' 0]
['20180806' '1_20180806_frames000003' '1' 0]
['20180806' '1_20180806_frames000004' '1' 0]
['20180806' '1_20180806_frames000005' '1' 0]
['20180806' '1_20180806_frames000006' '1' 0]
['20180806' '1_20180806_frames000007' '1' 0]
['20180806' '1_20180806_frames000008' '1' 0]
['20180806' '1_20180806_frames000009' '1' 0]
['20180806' '1_20180806_frames000010' '1' 0]
['20180806' '1_20180806_frames000011' '1' 0]
['20180806' '1_20180806_frames000012' '1' 0]
['20180806' '1_20180806_frames000013' '1' 0]
['20180806' '1_20180806_frames000014' '1' 0]
['20180806' '1_20180806_frames000015' '1' 0]
['20180806' '1_20180806_frames000016' '1' 0]
['20180806' '1_20180806_frames000017' '1' 0]
['20180806' '1_20180806_frames000018' '1' 0]
['20180806' '1_20180806_frames000019' '1' 0]
['20180806' '1_20180806_frames000020' '1' 0]
['20180806' '1_20180806_frames000021' '1' 0]
['20180806

['20180806' '1_20180806_frames000200' '1' 0]
['20180806' '1_20180806_frames000201' '1' 0]
['20180806' '1_20180806_frames000202' '1' 0]
['20180806' '1_20180806_frames000203' '1' 0]
['20180806' '1_20180806_frames000204' '1' 0]
['20180806' '1_20180806_frames000205' '1' 0]
['20180806' '1_20180806_frames000206' '1' 0]
['20180806' '1_20180806_frames000207' '1' 0]
['20180806' '1_20180806_frames000208' '1' 0]
['20180806' '1_20180806_frames000209' '1' 0]
['20180806' '1_20180806_frames000210' '1' 0]
['20180806' '1_20180806_frames000211' '1' 0]
['20180806' '1_20180806_frames000212' '1' 0]
['20180806' '1_20180806_frames000213' '1' 0]
['20180806' '1_20180806_frames000214' '1' 0]
['20180806' '1_20180806_frames000215' '1' 0]
['20180806' '1_20180806_frames000216' '1' 0]
['20180806' '1_20180806_frames000217' '1' 0]
['20180806' '1_20180806_frames000218' '1' 0]
['20180806' '1_20180806_frames000219' '1' 0]
['20180806' '1_20180806_frames000220' '1' 0]
['20180806' '1_20180806_frames000221' '1' 0]
['20180806

['20180806' '1_20180806_frames000418' '1' 0]
['20180806' '1_20180806_frames000419' '1' 0]
['20180806' '1_20180806_frames000420' '1' 0]
['20180806' '1_20180806_frames000421' '1' 0]
['20180806' '1_20180806_frames000422' '1' 0]
['20180806' '1_20180806_frames000423' '1' 0]
['20180806' '1_20180806_frames000424' '1' 0]
['20180806' '1_20180806_frames000425' '1' 0]
['20180806' '1_20180806_frames000426' '1' 0]
['20180806' '1_20180806_frames000427' '1' 0]
['20180806' '1_20180806_frames000428' '1' 0]
['20180806' '1_20180806_frames000429' '1' 0]
['20180806' '1_20180806_frames000430' '1' 0]
['20180806' '1_20180806_frames000431' '1' 0]
['20180806' '1_20180806_frames000432' '1' 0]
['20180806' '1_20180806_frames000433' '1' 0]
['20180806' '1_20180806_frames000434' '1' 0]
['20180806' '1_20180806_frames000435' '1' 0]
['20180806' '1_20180806_frames000436' '1' 0]
['20180806' '1_20180806_frames000437' '1' 0]
['20180806' '1_20180806_frames000438' '1' 0]
['20180806' '1_20180806_frames000439' '1' 0]
['20180806

['20180806' '1_20180806_frames000657' '1' 0]
['20180806' '1_20180806_frames000658' '1' 0]
['20180806' '1_20180806_frames000659' '1' 0]
['20180806' '1_20180806_frames000660' '1' 0]
['20180806' '1_20180806_frames000661' '1' 0]
['20180806' '1_20180806_frames000662' '1' 0]
['20180806' '1_20180806_frames000663' '1' 0]
['20180806' '1_20180806_frames000664' '1' 0]
['20180806' '1_20180806_frames000665' '1' 0]
['20180806' '1_20180806_frames000666' '1' 0]
['20180806' '1_20180806_frames000667' '1' 0]
['20180806' '1_20180806_frames000668' '1' 0]
['20180806' '1_20180806_frames000669' '1' 0]
['20180806' '1_20180806_frames000670' '1' 0]
['20180806' '1_20180806_frames000671' '1' 0]
['20180806' '1_20180806_frames000672' '1' 0]
['20180806' '1_20180806_frames000673' '1' 0]
['20180806' '1_20180806_frames000674' '1' 0]
['20180806' '1_20180806_frames000675' '1' 0]
['20180806' '1_20180806_frames000676' '1' 0]
['20180806' '1_20180806_frames000677' '1' 0]
['20180806' '1_20180806_frames000678' '1' 0]
['20180806

['20180806' '1_20180806_frames000907' '1' 0]
['20180806' '1_20180806_frames000908' '1' 0]
['20180806' '1_20180806_frames000909' '1' 0]
['20180806' '1_20180806_frames000911' '1' 1]
['20180806' '1_20180806_frames000912' '1' 1]
['20180806' '1_20180806_frames000913' '1' 1]
['20180806' '1_20180806_frames000914' '1' 1]
['20180806' '1_20180806_frames000915' '1' 1]
['20180806' '1_20180806_frames000916' '1' 1]
['20180806' '1_20180806_frames000917' '1' 1]
['20180806' '1_20180806_frames000918' '1' 1]
['20180806' '1_20180806_frames000919' '1' 1]
['20180806' '1_20180806_frames000920' '1' 1]
['20180806' '1_20180806_frames000921' '1' 1]
['20180806' '1_20180806_frames000922' '1' 1]
['20180806' '1_20180806_frames000923' '1' 1]
['20180806' '1_20180806_frames000924' '1' 1]
['20180806' '1_20180806_frames000925' '1' 1]
['20180806' '1_20180806_frames000926' '1' 1]
['20180806' '1_20180806_frames000927' '1' 1]
['20180806' '1_20180806_frames000928' '1' 1]
['20180806' '1_20180806_frames000929' '1' 1]
['20180806

...

['20180920' '6_20180920_frames000159' '6' 0]
['20180920' '6_20180920_frames000160' '6' 0]
['20180920' '6_20180920_frames000161' '6' 0]
['20180920' '6_20180920_frames000162' '6' 0]
['20180920' '6_20180920_frames000163' '6' 0]
['20180920' '6_20180920_frames000164' '6' 0]
['20180920' '6_20180920_frames000165' '6' 0]
['20180920' '6_20180920_frames000166' '6' 0]
['20180920' '6_20180920_frames000167' '6' 0]
['20180920' '6_20180920_frames000168' '6' 0]
['20180920' '6_20180920_frames000169' '6' 0]
['20180920' '6_20180920_frames000170' '6' 0]
['20180920' '6_20180920_frames000171' '6' 0]
['20180920' '6_20180920_frames000172' '6' 0]
['20180920' '6_20180920_frames000173' '6' 0]
['20180920' '6_20180920_frames000174' '6' 0]
['20180920' '6_20180920_frames000175' '6' 0]
['20180920' '6_20180920_frames000176' '6' 0]
['20180920' '6_20180920_frames000177' '6' 0]
['20180920' '6_20180920_frames000178' '6' 0]
['20180920' '6_20180920_frames000179' '6' 0]
['20180920' '6_20180920_frames000180' '6' 0]
['20180920

['20180920' '6_20180920_frames000378' '6' 0]
['20180920' '6_20180920_frames000379' '6' 0]
['20180920' '6_20180920_frames000380' '6' 0]
['20180920' '6_20180920_frames000381' '6' 0]
['20180920' '6_20180920_frames000382' '6' 0]
['20180920' '6_20180920_frames000383' '6' 0]
['20180920' '6_20180920_frames000384' '6' 0]
['20180920' '6_20180920_frames000385' '6' 0]
['20180920' '6_20180920_frames000386' '6' 0]
['20180920' '6_20180920_frames000387' '6' 0]
['20180920' '6_20180920_frames000388' '6' 0]
['20180920' '6_20180920_frames000389' '6' 0]
['20180920' '6_20180920_frames000390' '6' 0]
['20180920' '6_20180920_frames000391' '6' 0]
['20180920' '6_20180920_frames000392' '6' 0]
['20180920' '6_20180920_frames000393' '6' 0]
['20180920' '6_20180920_frames000394' '6' 0]
['20180920' '6_20180920_frames000395' '6' 0]
['20180920' '6_20180920_frames000396' '6' 0]
['20180920' '6_20180920_frames000397' '6' 0]
['20180920' '6_20180920_frames000398' '6' 0]
['20180920' '6_20180920_frames000399' '6' 0]
['20180920

['20180920' '7_20180920_frames000040' '7' 0]
['20180920' '7_20180920_frames000041' '7' 0]
['20180920' '7_20180920_frames000042' '7' 0]
['20180920' '7_20180920_frames000043' '7' 0]
['20180920' '7_20180920_frames000044' '7' 0]
['20180920' '7_20180920_frames000045' '7' 0]
['20180920' '7_20180920_frames000046' '7' 0]
['20180920' '7_20180920_frames000047' '7' 0]
['20180920' '7_20180920_frames000048' '7' 0]
['20180920' '7_20180920_frames000049' '7' 0]
['20180920' '7_20180920_frames000050' '7' 0]
['20180920' '7_20180920_frames000051' '7' 0]
['20180920' '7_20180920_frames000052' '7' 0]
['20180920' '7_20180920_frames000053' '7' 0]
['20180920' '7_20180920_frames000054' '7' 0]
['20180920' '7_20180920_frames000055' '7' 0]
['20180920' '7_20180920_frames000056' '7' 0]
['20180920' '7_20180920_frames000057' '7' 0]
['20180920' '7_20180920_frames000058' '7' 0]
['20180920' '7_20180920_frames000059' '7' 0]
['20180920' '7_20180920_frames000060' '7' 0]
['20180920' '7_20180920_frames000061' '7' 0]
['20180920

['20180920' '7_20180920_frames000283' '7' 1]
['20180920' '7_20180920_frames000284' '7' 1]
['20180920' '7_20180920_frames000285' '7' 0]
['20180920' '7_20180920_frames000286' '7' 0]
['20180920' '7_20180920_frames000287' '7' 0]
['20180920' '7_20180920_frames000288' '7' 0]
['20180920' '7_20180920_frames000289' '7' 0]
['20180920' '7_20180920_frames000290' '7' 0]
['20180920' '7_20180920_frames000291' '7' 0]
['20180920' '7_20180920_frames000292' '7' 0]
['20180920' '7_20180920_frames000293' '7' 0]
['20180920' '7_20180920_frames000294' '7' 0]
['20180920' '7_20180920_frames000295' '7' 0]
['20180920' '7_20180920_frames000296' '7' 0]
['20180920' '7_20180920_frames000297' '7' 0]
['20180920' '7_20180920_frames000298' '7' 0]
['20180920' '7_20180920_frames000299' '7' 0]
['20180920' '7_20180920_frames000300' '7' 0]
['20180920' '7_20180920_frames000301' '7' 0]
['20180920' '7_20180920_frames000302' '7' 0]
['20180920' '7_20180920_frames000303' '7' 0]
['20180920' '7_20180920_frames000304' '7' 0]
['20180920

['20180920' '7_20180920_frames000517' '7' 0]
['20180920' '7_20180920_frames000518' '7' 0]
['20180920' '7_20180920_frames000519' '7' 0]
['20180920' '7_20180920_frames000520' '7' 0]
['20180920' '7_20180920_frames000521' '7' 0]
['20180920' '7_20180920_frames000522' '7' 0]
['20180920' '7_20180920_frames000523' '7' 0]
['20180920' '7_20180920_frames000524' '7' 0]
['20180920' '7_20180920_frames000525' '7' 0]
['20180920' '7_20180920_frames000526' '7' 0]
['20180920' '7_20180920_frames000527' '7' 0]
['20180920' '7_20180920_frames000528' '7' 0]
['20180920' '7_20180920_frames000529' '7' 0]
['20180920' '7_20180920_frames000530' '7' 0]
['20180920' '7_20180920_frames000531' '7' 0]
['20180920' '7_20180920_frames000532' '7' 0]
['20180920' '7_20180920_frames000533' '7' 0]
['20180920' '7_20180920_frames000534' '7' 0]
['20180920' '7_20180920_frames000535' '7' 0]
['20180920' '7_20180920_frames000536' '7' 0]
['20180920' '7_20180920_frames000537' '7' 0]
['20180920' '7_20180920_frames000538' '7' 0]
['20180920

In [47]:
labeling = pd.DataFrame(hh_dict, index =range(len(hh_dict['img_name'])))

In [48]:
pd.DataFrame.to_csv(labeling, label_path, index=False)